In [1]:
import pandas as pd
import numpy as np

train_X = pd.read_csv('data/train.csv')
test_X = pd.read_csv('data/test.csv')

train_X.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


## Null Data check

In [2]:
columns = train_X.columns
for column in columns:
    null_check= train_X[column].isnull().sum()/train_X[column].shape[0]
    print(column, null_check)

PassengerId 0.0
Survived 0.0
Pclass 0.0
Name 0.0
Sex 0.0
Age 0.19865319865319866
SibSp 0.0
Parch 0.0
Ticket 0.0
Fare 0.0
Cabin 0.7710437710437711
Embarked 0.002244668911335578


In [3]:
train_X["Embarked"].fillna('S',inplace=True)
train_X["Embarked"].isnull().sum()

0

### regex 사용하여 initial name 알아내기

In [4]:
train_X['Initial'] = train_X["Name"].str.extract('([A-Za-z]+)\.')
test_X['Initial'] = test_X["Name"].str.extract('([A-Za-z]+)\.')
pd.crosstab(train_X['Initial'], train_X['Sex']).T

Initial,Capt,Col,Countess,Don,Dr,Jonkheer,Lady,Major,Master,Miss,Mlle,Mme,Mr,Mrs,Ms,Rev,Sir
Sex,,,,,,,,,,,,,,,,,
female,0,0,1,0,1,0,1,0,0,182,2,1,0,125,1,0,0
male,1,2,0,1,6,1,0,2,40,0,0,0,517,0,0,6,1


In [5]:
train_X['Initial'].replace(['Mlle','Mme','Ms','Dr','Major','Lady','Countess','Jonkheer','Col','Rev','Capt','Sir','Don', 'Dona'],
                        ['Miss','Miss','Miss','Mr','Mr','Mrs','Mrs','Other','Other','Other','Mr','Mr','Mr', 'Mr'],inplace=True)

test_X['Initial'].replace(['Mlle','Mme','Ms','Dr','Major','Lady','Countess','Jonkheer','Col','Rev','Capt','Sir','Don', 'Dona'],
                        ['Miss','Miss','Miss','Mr','Mr','Mrs','Mrs','Other','Other','Other','Mr','Mr','Mr', 'Mr'],inplace=True)

In [6]:
train_X.groupby('Initial').mean()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
Initial,,,,,,,
Master,414.975000,0.575000,2.625000,4.574167,2.300000,1.375000,34.703125
Miss,411.741935,0.704301,2.284946,21.860000,0.698925,0.537634,43.937188
Mr,455.880907,0.162571,2.381853,32.739609,0.293006,0.151229,24.939822
Mrs,456.393701,0.795276,1.984252,35.981818,0.692913,0.818898,45.420603
Other,564.444444,0.111111,1.666667,45.888889,0.111111,0.111111,19.102778


### Age null data 처리

In [7]:
train_X.loc[(train_X.Age.isnull())& (train_X.Initial =='Mr'),'Age'] =33
train_X.loc[(train_X.Age.isnull())&(train_X.Initial=='Mrs'),'Age'] = 36
train_X.loc[(train_X.Age.isnull())&(train_X.Initial=='Master'),'Age'] = 5
train_X.loc[(train_X.Age.isnull())&(train_X.Initial=='Miss'),'Age'] = 22
train_X.loc[(train_X.Age.isnull())&(train_X.Initial=='Other'),'Age'] = 46

test_X.loc[(test_X.Age.isnull())&(test_X.Initial=='Mr'),'Age'] = 33
test_X.loc[(test_X.Age.isnull())&(test_X.Initial=='Mrs'),'Age'] = 36
test_X.loc[(test_X.Age.isnull())&(test_X.Initial=='Master'),'Age'] = 5
test_X.loc[(test_X.Age.isnull())&(test_X.Initial=='Miss'),'Age'] = 22
test_X.loc[(test_X.Age.isnull())&(test_X.Initial=='Other'),'Age'] = 46

In [8]:
train_X['Initial'] = train_X['Initial'].map({'Master': 0, 'Miss': 1, 'Mr': 2, 'Mrs': 3, 'Other': 4})
test_X['Initial'] = test_X['Initial'].map({'Master': 0, 'Miss': 1, 'Mr': 2, 'Mrs': 3, 'Other': 4})

# one hot enoding  initial name
train_X = pd.get_dummies(train_X, columns=['Initial'], prefix='Initial')
test_X = pd.get_dummies(test_X, columns=['Initial'], prefix='Initial')

## categorize age by dividing 10 year

In [9]:
def category_age(x):
    if x < 10:
        return 0
    elif x < 20:
        return 1
    elif x< 30:
        return 2
    elif x < 40:
        return 3
    elif x< 50:
        return 4
    elif x < 60:
        return 5
    elif x< 70:
        return 6
    else:
        return 7

In [10]:
train_X["Age_category"]=train_X["Age"].apply(category_age)
test_X["Age_category"] = test_X["Age"].apply(category_age)

### Fare column EDA

In [11]:
train_X.loc[train_X.Fare.isnull(), 'Fare'] = train_X['Fare'].mean()
test_X.loc[test_X.Fare.isnull(), 'Fare'] = test_X['Fare'].mean()

train_X['Fare'] = train_X['Fare'].map(lambda i: np.log(i) if i > 0 else 0)
test_X['Fare'] = test_X['Fare'].map(lambda i: np.log(i) if i > 0 else 0)

### change string to categorical data (각 피쳐들을 수치로 바꿈)

In [12]:
# Embarked Feature

train_X["Embarked"] = train_X["Embarked"].map({'C':0, 'Q':1,'S':2})
test_X["Embarked"] = test_X["Embarked"].map({'C':0, 'Q':1,'S':2})

In [13]:
# Sex Feature
train_X["Sex"] = train_X["Sex"].map({'male':1, 'female':0})
test_X["Sex"] = test_X["Sex"].map({'male':1, 'female':0})

### one hot encoding

In [14]:
train_embarked= pd.get_dummies(train_X["Embarked"], prefix="Embarked")
train_X = pd.concat([train_X, train_embarked], axis = 1)

test_embarked= pd.get_dummies(test_X["Embarked"], prefix="Embarked")
test_X = pd.concat([test_X, test_embarked], axis = 1)

In [15]:

train_X["family_cnt"] = train_X["SibSp"] + train_X["Parch"]
test_X["family_cnt"] = test_X["SibSp"] + test_X["Parch"]
train_X.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,...,Initial_0,Initial_1,Initial_2,Initial_3,Initial_4,Age_category,Embarked_0,Embarked_1,Embarked_2,family_cnt
0,1,0,3,"Braund, Mr. Owen Harris",1,22.0,1,0,A/5 21171,1.981001,...,0,0,1,0,0,2,0,0,1,1
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,38.0,1,0,PC 17599,4.266662,...,0,0,0,1,0,3,1,0,0,1
2,3,1,3,"Heikkinen, Miss. Laina",0,26.0,0,0,STON/O2. 3101282,2.070022,...,0,1,0,0,0,2,0,0,1,0
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,35.0,1,0,113803,3.972177,...,0,0,0,1,0,3,0,0,1,1
4,5,0,3,"Allen, Mr. William Henry",1,35.0,0,0,373450,2.085672,...,0,0,1,0,0,3,0,0,1,0


In [16]:
train_X.to_csv('./refine_data/train.csv')
test_X.to_csv('./refine_data/test.csv')

### Train

1. 데이터를 활용해 머신러닝 알고리즘을 학습시켜본다. 
2. 데이터를 label(맞춰야하는 정답)과 Feature(맞춰야하는데 도움이 되는 값)들로 나눈다. 

3. X_train: train 데이터의 feature, 줄여서 X_train이라고 지정.
4. X_test: test 데이터의 feature, 마찬가지로 줄여서 X_test라고 지정.
5. y_train: train 데이터의 label, 마찬가지로 줄여서 y_train이라 지정

In [ ]:
from sklearn.model_selection import train_test_split

features =["Pclass","Sex","Age_category","Embarked_0","Embarked_1","Embarked_2","family_cnt"]
label = "Survived"

x_train, x_test, y_train, y_test= train_test_split(train_X[features], train_X[label],test_size= 0.3,random_state= 37)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

model = RandomForestClassifier()
model.fit(x_train,y_train)

In [ ]:
prediction = model.predict(x_test)

In [ ]:
metrics.accuracy_score(prediction, y_test)

### Feature Importance
- 만든 모델이 어떤 feature에 영향을 많이 받았는지 확인 할 수 있다. 


In [ ]:
from pandas import Series
feature_importance = model.feature_importances_
Series_feat_im = Series(feature_importance, index=test_X[features].columns)

In [ ]:
import matplotlib.pyplot as plt
Series_feat_im.sort_values(ascending=True).plot.barh()
plt.xlabel("Feature Importance")
plt.ylabel("Feature")
plt.show()

In [ ]:
submission = pd.read_csv('data/sample_submission.csv')
submission.head()

In [ ]:
prediction =model.predict(test_X[features])
submission["Survived"] = prediction

In [ ]:
submission.to_csv('./first_submission.csv',index=False)